In [8]:
import numpy as np
import nltk
import string
import requests
import re
import glob
import os, sys
import datetime
from nltk.corpus import wordnet as wn
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import pandas as pd
import xlrd
import csv
import time
import dask
import dask.dataframe as dd
from bs4 import BeautifulSoup
from sec_edgar_downloader import Downloader
from sklearn.feature_extraction.text import TfidfVectorizer
import shutil
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
import nltk
from data_preparation import clean_ticker, download_to_item1
from document_similarity import preprocess, similarity_calculation, last_3y_similarity, last_5y_similarity
nltk.download('stopwords')
dl = Downloader("/home/hongzhuoqiao/10K_Projects/sec_filings")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/hongzhuoqiao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
company_df = pd.read_csv("/home/hongzhuoqiao/10K_Projects/sec_filings/multiple_observations.txt", sep='\t')
tc_df = company_df[['cusip','tic']].drop_duplicates()
tc_df['ticker'] = tc_df['tic'].apply(clean_ticker)
tc_df.to_csv("/home/hongzhuoqiao/10K_Projects/sec_filings/tc.csv")
tc_df.head()

,cusip,tic,ticker
0,000361105,AIR,AIR
13,000886309,ADCT.1,ADCT
17,001058205,IWKS,IWKS
19,020813101,ALO.2,ALO
20,02376R102,AAL,AAL


In [9]:
# Get companies in the original list
result_df = pd.read_csv("result.csv", index_col=0)
print("{} company names provided in the original list.\n".format(len(result_df)))

# Get companies have 10k files extracted
havefile_df = result_df[result_df['totals']!=0]
print("{} company have 10k files extracted.\n".format(len(havefile_df)))

# Get companies have 10k file and no error flag (not necessary error free
flag_free_df = havefile_df[havefile_df['errors']==0]
print("{} companies have no error flag. (Not neccessary error free.)\n".format(len(flag_free_df)))
flag_free_df.head()

19825 company names provided in the original list.

7842 company have 10k files extracted.

5697 companies have no error flag. (Not neccessary error free.)



,cusip,tic,ticker,extraction_info,totals,extracted,errors
0,000361105,AIR,AIR,"26, 26, 0, []",26,26,0
13,000886309,ADCT.1,ADCT,"18, 18, 0, []",18,18,0
19,020813101,ALO.2,ALO,"14, 14, 0, []",14,14,0
32,125141101,CECE,CECE,"19, 19, 0, []",19,19,0
81,723484101,PNW,PNW,"27, 27, 0, []",27,27,0


In [10]:
# detect and correct extraction
# TODO: methods to detect abnormality in extracted item1 files (using similarity score, length)


In [80]:
# Generate 3y_sim files
# Last_3y_sim calculation from existing sim_matrix
sim_matrix_files_path = "/home/hongzhuoqiao/10K_Projects/sec_filings/tfidf_similarity/sim_matrix/*.csv"
sim_matrix_files = glob.glob(sim_matrix_files_path)
sim_3y_df_list =[]

for file in sim_matrix_files:
    filename = os.path.basename(file)
    tickername = filename[:-11]
    sim_df = pd.read_csv(file, index_col=0)
    sim_3y_df = last_3y_similarity(sim_df,tickername)
    sim_3y_df_list.append(sim_3y_df)
    print(">>> {} sim_3y appended".format(tickername))
    sim_3y_df.to_csv("/home/hongzhuoqiao/10K_Projects/sec_filings/tfidf_similarity/sim_3y/"+tickername+"_3y.csv")
    print(">>> Got 3y similarity pair for {}".format(tickername))

sim_3y_df = pd.concat(sim_3y_df_list, axis=1)
sim_3y_df.head()

rity pair for SOI
>>> KYAK sim_3y appended
>>> Got 3y similarity pair for KYAK
>>> CPTA sim_3y appended
>>> Got 3y similarity pair for CPTA
>>> EQT sim_3y appended
>>> Got 3y similarity pair for EQT
>>> BR sim_3y appended
>>> Got 3y similarity pair for BR
>>> TCBI sim_3y appended
>>> Got 3y similarity pair for TCBI
>>> RCHA sim_3y appended
>>> Got 3y similarity pair for RCHA
>>> NBTF sim_3y appended
>>> Got 3y similarity pair for NBTF
>>> SWX sim_3y appended
>>> Got 3y similarity pair for SWX
>>> KED sim_3y appended
>>> Got 3y similarity pair for KED
>>> AMCO sim_3y appended
>>> Got 3y similarity pair for AMCO
>>> VLNCQ sim_3y appended
>>> Got 3y similarity pair for VLNCQ
>>> CVGW sim_3y appended
>>> Got 3y similarity pair for CVGW
>>> HCFT sim_3y appended
>>> Got 3y similarity pair for HCFT
>>> SUAI sim_3y appended
>>> Got 3y similarity pair for SUAI
>>> PACI sim_3y appended
>>> Got 3y similarity pair for PACI
>>> TFS sim_3y appended
>>> Got 3y similarity pair for TFS
>>> BAX sim_3y a

,MDXG,CFNL,HRLY,SWCH,IPSU,PLAG,LORL,ANSW,WPG,EDUC,...,LITE,AROC,CVD,MHGC,OCR,CSGS,ARCH,NGSX,RNOW,SFS
1997,NaN,NaN,NaN,NaN,0.956083,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998,NaN,NaN,0.912484,NaN,0.925148,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.949402,NaN,NaN,NaN,NaN,NaN
1999,NaN,NaN,0.928833,NaN,0.950455,NaN,0.831361,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.957610,NaN,NaN,NaN,NaN,NaN
2000,NaN,NaN,0.750240,NaN,0.976917,NaN,0.872993,NaN,NaN,0.919292,...,NaN,NaN,0.821110,NaN,0.965917,0.920033,NaN,NaN,NaN,NaN
2001,NaN,NaN,0.865913,NaN,0.866062,NaN,0.911002,NaN,NaN,0.958608,...,NaN,NaN,0.848734,NaN,0.897582,0.947296,0.728552,NaN,NaN,NaN


In [7]:
# Generate 5y_sim files
# Last_5y_sim calculation from existing sim_matrix
sim_matrix_files_path = "/home/hongzhuoqiao/10K_Projects/sec_filings/tfidf_similarity/sim_matrix/*.csv"
sim_matrix_files = glob.glob(sim_matrix_files_path)
sim_5y_df_list =[]

for file in sim_matrix_files:
    filename = os.path.basename(file)
    tickername = filename[:-11]
    sim_df = pd.read_csv(file, index_col=0)
    sim_5y_df = last_5y_similarity(sim_df,tickername)
    sim_5y_df.to_csv("/home/hongzhuoqiao/10K_Projects/sec_filings/tfidf_similarity/sim_5y/"+tickername+"_5y.csv")
    print("\n>>> Saved 5y similarity pair for {}".format(tickername))

    sim_5y_df_list.append(sim_5y_df)
    print(">>> {} sim_5y appended".format(tickername))
    
sim_5y_df = pd.concat(sim_5y_df_list, axis=1)
sim_5y_df.head()

rity pair for TFS
>>> TFS sim_5y appended

>>> Saved 5y similarity pair for BAX
>>> BAX sim_5y appended

>>> Saved 5y similarity pair for PAYS
>>> PAYS sim_5y appended

>>> Saved 5y similarity pair for VITL
>>> VITL sim_5y appended

>>> Saved 5y similarity pair for ALTH
>>> ALTH sim_5y appended

>>> Saved 5y similarity pair for ISYS
>>> ISYS sim_5y appended

>>> Saved 5y similarity pair for FLTX
>>> FLTX sim_5y appended

>>> Saved 5y similarity pair for MCBF
>>> MCBF sim_5y appended

>>> Saved 5y similarity pair for UMRX
>>> UMRX sim_5y appended

>>> Saved 5y similarity pair for ZCOR
>>> ZCOR sim_5y appended

>>> Saved 5y similarity pair for CHZS
>>> CHZS sim_5y appended

>>> Saved 5y similarity pair for XOM
>>> XOM sim_5y appended

>>> Saved 5y similarity pair for ABR
>>> ABR sim_5y appended

>>> Saved 5y similarity pair for MACK
>>> MACK sim_5y appended

>>> Saved 5y similarity pair for IRTC
>>> IRTC sim_5y appended

>>> Saved 5y similarity pair for TNF
>>> TNF sim_5y appended

>>> S

,MDXG,CFNL,HRLY,SWCH,IPSU,PLAG,LORL,ANSW,WPG,EDUC,...,LITE,AROC,CVD,MHGC,OCR,CSGS,ARCH,NGSX,RNOW,SFS
1999,NaN,NaN,NaN,NaN,0.931265,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000,NaN,NaN,0.731821,NaN,0.948314,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.944469,NaN,NaN,NaN,NaN,NaN
2001,NaN,NaN,0.850927,NaN,0.865451,NaN,0.821383,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.876367,NaN,NaN,NaN,NaN,NaN
2002,NaN,NaN,0.724937,NaN,0.880448,NaN,0.872888,NaN,NaN,0.949164,...,NaN,NaN,0.826216,NaN,0.912329,0.890936,NaN,NaN,NaN,NaN
2003,NaN,NaN,0.779510,NaN,0.900157,NaN,0.919951,NaN,NaN,0.960047,...,NaN,NaN,0.858767,NaN,0.938945,0.830598,0.826603,NaN,NaN,NaN


In [8]:
sim_5y_df.head()

,MDXG,CFNL,HRLY,SWCH,IPSU,PLAG,LORL,ANSW,WPG,EDUC,...,LITE,AROC,CVD,MHGC,OCR,CSGS,ARCH,NGSX,RNOW,SFS
1999,NaN,NaN,NaN,NaN,0.931265,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2000,NaN,NaN,0.731821,NaN,0.948314,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.944469,NaN,NaN,NaN,NaN,NaN
2001,NaN,NaN,0.850927,NaN,0.865451,NaN,0.821383,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.876367,NaN,NaN,NaN,NaN,NaN
2002,NaN,NaN,0.724937,NaN,0.880448,NaN,0.872888,NaN,NaN,0.949164,...,NaN,NaN,0.826216,NaN,0.912329,0.890936,NaN,NaN,NaN,NaN
2003,NaN,NaN,0.779510,NaN,0.900157,NaN,0.919951,NaN,NaN,0.960047,...,NaN,NaN,0.858767,NaN,0.938945,0.830598,0.826603,NaN,NaN,NaN


In [9]:
#sim_3y_df.T.merge(result_df[['cusip','ticker']], right_on='ticker', left_index=True, how='left').to_csv('sim_3y_03181800.csv')
#sim_5y_df.T.merge(result_df[['cusip','ticker']], right_on='ticker', left_index=True, how='left').to_csv('sim_5y_03191500.csv')

In [11]:
#sim_5y_df.T.merge(result_df[['cusip','ticker']], right_on='ticker', left_index=True, how='left')